In [18]:
import numpy_financial as npf

cashflows = [
    -50000,
    -42000,
    -40000,
    -40000,
    -43000,
    -38500,
    -50000,
    -50000,
    -50000,
    650000,
    -50000,
    -50000,
    -50000,
    -50000,
]

# 1. Net Profit/Loss
net_profit = sum(cashflows)
print(f"Net Profit/Loss: {net_profit}")

# 2. Modified Internal Rate of Return (MIRR)
# Since IRR is undefined for total loss, we calculate MIRR
# Finance Rate = 0% (Cost of cash), Reinvest Rate = 0%
pos_flows = [c for c in cashflows if c > 0]
neg_flows = [c for c in cashflows if c < 0]

fv_pos = sum(pos_flows)  # 400,000
pv_neg = abs(sum(neg_flows))  # 603,500
n_periods = len(cashflows)

# MIRR Formula: (FV / PV)^(1/n) - 1
mirr_periodic = (fv_pos / pv_neg) ** (1 / n_periods) - 1
mirr_annual = (1 + mirr_periodic) ** 2 - 1  # Annualized

print(f"Annualized Return %: {mirr_annual:.2%}")


Net Profit/Loss: 46500
Annualized Return %: 1.07%


In [101]:
flows = [
    -50000,
    -42000,
    -40000,
    -40000,
    -43000,
    -38500,
    -42000,
    -45000,
    -47000,
    650000,
    -50000,
    -50000,
    -50000,
    -50000,
]
np.sum(flows)

np.int64(62500)

In [114]:
npf.npv(0.07, flows)


np.float64(-39024.794485865204)

In [122]:
((1+npf.irr(flows))**(2))-1

0.07571925976983729

In [66]:
pre=[-50000, -42000, -40000, -40000, -43000, -38500, -42000, 
    -45000, -47000]
post=[-50000, -50000, -50000, -50000]
npf.mirr(pre + [400000] + post, finance_rate=0, reinvest_rate=0.012)

np.float64(-0.025567278281945383)

In [58]:
import numpy_financial as npf
import math

def analyze_chit_loss():
    # Your Cash Flows
    flows = [
        -50000, -42000, -40000, -40000, -43000, -38500, -42000, 
        -45000, -47000, 500000, -50000, -50000, -50000, -50000
    ]

    # 1. Try calculating IRR
    irr_result = npf.irr(flows)

    print(f"{'Metric':<20} | {'Value':<15}")
    print("-" * 40)

    # Check if result is valid (not nan)
    if math.isnan(irr_result):
        print(f"{'IRR Result':<20} | {'No Solution (nan)':<15}")
        print("-" * 40)
        print("EXPLANATION:")
        print("The calculation failed because the cash flows are 'Non-Conventional'.")
        print("You have multiple sign changes (Out -> In -> Out) and")
        print("the total outflows significantly exceed the inflows.")
    else:
        print(f"{'IRR':<20} | {irr_result:.2%}")
        print((irr_result)**(1/4))

    # 2. Calculate Absolute Net Loss (Accounting Profit/Loss)
    # Source: Module 2 mentions Profit Maximization considers absolute resources [5].
    total_paid = sum(f for f in flows if f < 0)
    total_received = sum(f for f in flows if f > 0)
    net_position = sum(flows)

    print("-" * 40)
    print(f"{'Total Invested':<20} | {abs(total_paid):,.2f}")
    print(f"{'Total Received':<20} | {total_received:,.2f}")
    print(f"{'Net Absolute Loss':<20} | {net_position:,.2f}")
    
    # 3. Calculate ROI (Return on Investment) - Simple Method
    # This is a basic percentage of how much you lost relative to what you put in.
    roi = (net_position / abs(total_paid)) * 100
    print(f"{'Simple ROI':<20} | {roi:.2f}%")

analyze_chit_loss()

Metric               | Value          
----------------------------------------
IRR                  | -9.98%
(0.3974655214723401+0.3974655214723401j)
----------------------------------------
Total Invested       | 587,500.00
Total Received       | 500,000.00
Net Absolute Loss    | -87,500.00
Simple ROI           | -14.89%


In [5]:
# Tests for mean() and borrower_rate
import unittest


class TestMean(unittest.TestCase):
    def test_simple_lists(self):
        self.assertAlmostEqual(mean([1, 2, 3]), 2.0)
        self.assertAlmostEqual(mean([10, 20, 30, 40]), 25.0)

    def test_single_value(self):
        self.assertAlmostEqual(mean([42]), 42.0)

    def test_raises_on_empty(self):
        with self.assertRaises(ValueError):
            mean([])


class TestBorrowerRate(unittest.TestCase):
    def test_no_solution_for_given_chit_case(self):
        """For the provided 7-year cash flows, no real-valued rate exists."""
        cashflows = [
            -50000,
            -42000,
            -40000,
            -40000,
            -43000,
            -38500,
            -50000,
            -50000,
            -50000,
            400000,
            -50000,
            -50000,
            -50000,
            -50000,
        ]
        receipt_idx = 9
        with self.assertRaises(RuntimeError):
            borrower_rate(cashflows, receipt_idx, period_years=0.5)

    def test_positive_rate_for_synthetic_loan_like_case(self):
        """A simple loan-like cash-flow pattern should yield a positive rate."""
        # Borrower receives 10,000 at t=0 and repays 5,500 at 0.5y and 5,500 at 1.0y
        cashflows = [
            10000.0,
            -5500.0,
            -5500.0,
        ]
        receipt_idx = 0  # chit/loan received immediately
        r = borrower_rate(cashflows, receipt_idx, period_years=0.5)
        # Expect a positive rate on the order of ~9-11% p.a.
        self.assertGreater(r, 0.0)
        self.assertLess(r, 0.2)

    def test_scale_invariance_on_synthetic_case(self):
        """Scaling all cash flows by a constant should not change the rate."""
        cashflows = [
            10000.0,
            -5500.0,
            -5500.0,
        ]
        receipt_idx = 0
        r1 = borrower_rate(cashflows, receipt_idx, period_years=0.5)
        scaled = [cf * 10 for cf in cashflows]
        r2 = borrower_rate(scaled, receipt_idx, period_years=0.5)
        self.assertAlmostEqual(r1, r2, places=8)


unittest.main(argv=[""], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK
